### 34.1 Metaclass

A metaclass is a class of a class. 
It allows you to customize class creation and behavior

What are the uses of metaclasses?
- automatically register a class
- validating class attributes and methods
- modify the class even before it is created
- enforce coding standards

### 34.2 Example

##### Creating a metaclass

In [19]:
class MyMeta(type):

    def __new__(cls, name, bases, dct):
        dct['class_id'] = name.upper()
        print('Creating a class: ')
        print(cls)
        print(name)
        print(bases)
        print(dct)
        return super().__new__(cls, name, bases, dct)
        

##### Using a metaclass

In [22]:
class MyClass(metaclass=MyMeta):

    def greet(self):
        return f"{self.class_id}"
    

Creating a class: 
<class '__main__.MyMeta'>
MyClass
()
{'__module__': '__main__', '__qualname__': 'MyClass', 'greet': <function MyClass.greet at 0x000001DCB4EB6B60>, 'class_id': 'MYCLASS'}


In [24]:
obj = MyClass()
obj.greet()

'MYCLASS'

What is __new__()?
Special method -static method
Creates a new instance of a class
Gets called before the __init__
Hence it can be used to customize the class creation itself
Usually used when you want to control how classes are created
Sometime you may want to inject some attributes, methods and validation logic in to the class

Let's try to use __new__ in a normal class

In [26]:
class MyClass:

    def __new__(cls):
        print("Creating the class...")
        instance = super().__new__(cls)
        return instance

    def __init__(self):
        print("Initialization starts...")

In [28]:
m = MyClass()

Creating the class...
Initialization starts...


### 34.3 Validating Attributes using Metaclasses

Let's say we want to make sure that the classes implement the required_attrs and it should be in the form of a list. In this case, we can create a metaclass that makes sure of this 

In [43]:
class ValidateAttrs(type):
    
    def __new__(cls, name, bases, dct):
        if 'required_attrs' not in dct:
            raise TypeError(f"Class {name} should implement required_attrs")
        if not isinstance(dct['required_attrs'], list):
            raise TypeError(f"required_attrs in Class {name} should be of type list")
        return super().__new__(cls, name, bases, dct)

In [51]:
# Implementing without required_attrs: it will not even allow your to create a class
class A(metaclass=ValidateAttrs):

    def __init__(self):
        print("Class created")

TypeError: Class A should implement required_attrs

In [47]:
# With required_attrs
class A(metaclass=ValidateAttrs):

    required_attrs = ["name", "email"]

    def __init__(self):
        print("Class created")

##### A much better class implementation with a purpose

In [96]:
class Person(metaclass=ValidateAttrs):

    required_attrs = ['name', 'email']

    def __init__(self, **kargs):

        missing = [attr for attr in Person.required_attrs if attr not in kargs]
        if missing:
            raise ValueError(f"Missing attributes: {missing}")

        for key, value in dict(kargs).items():
            setattr(self, key, value)

In [98]:
p = Person(name="Anil")

ValueError: Missing attributes: ['email']

In [100]:
p = Person(name="Anil", email="anil@ust.com")

In [102]:
p

##### The locals() Function

In [136]:
class Person(metaclass=ValidateAttrs):

    required_attrs = ['name', 'email']

    def __init__(self, **kargs):

        print('LOCALS: ', locals())
        
        missing = [attr for attr in Person.required_attrs if attr not in kargs]
        if missing:
            raise ValueError(f"Missing attributes: {missing}")

        for key, value in dict(kargs).items():
            setattr(self, key, value)

In [138]:
p = Person(name="Anil", email="anil@ust.com")

LOCALS:  {'self': <__main__.Person object at 0x000001DCB5D8EE10>, 'kargs': {'name': 'Anil', 'email': 'anil@ust.com'}}


### 34.4 Getting attributes of an object

In [104]:
dir(p)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'email',
 'name',
 'required_attrs']

In [106]:
p.__dict__

{'name': 'Anil', 'email': 'anil@ust.com'}

In [108]:
p1 = Person(name="Sunil", email="sunil@ust.com", salary="1000000 INR", location="Mumbai")

In [110]:
dir(p1)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'email',
 'location',
 'name',
 'required_attrs',
 'salary']

In [114]:
p1.salary, p1.location

('1000000 INR', 'Mumbai')

In [116]:
p1.__dict__

{'name': 'Sunil',
 'email': 'sunil@ust.com',
 'salary': '1000000 INR',
 'location': 'Mumbai'}

### 34.5 Problem Statement

Create a metaclass called EnforceFieldTypes that ensures any class using it must define a dictionary called field_types. This dictionary maps attribute names to their expected types (e.g., {"name": str, "age": int}).

The metaclass should inject a custom __init__ method into the class that:

    Accepts keyword arguments for all fields in field_types.
    Validates that each provided argument matches the expected type.
    Raises a TypeError if a field is missing or if the value type is incorrect.

Use this metaclass to implement a Student class.

#### Without injection of __init__

In [124]:
# Meta Class
class EnforceFieldTypes(type):

    def __new__(cls, name, bases, dct):

        # Extract field_types
        field_types = dct.get('field_types') # dct['field_types']
         
        # Validate the field_types
        if not isinstance(field_types, dict):
            raise TypeError(f"Class '{name}' must define 'field_types' dictionary")

        # We will also enforce __init__ to be defined in the class
        if '__init__' not in dct:
            raise TypeError(f"Class '{name}' must define '__init__' function")

        return super().__new__(cls, name, bases, dct)

In [126]:
class A(metaclass=EnforceFieldTypes):

    def print(self):
        print("This is a class")

TypeError: Class 'A' must define 'field_types' dictionary

In [129]:
# Let's add the field_types
class A(metaclass=EnforceFieldTypes):

    field_types = {'name':str, 'age':int}

    def print(self):
        print("This is a class")

TypeError: Class 'A' must define '__init__' function

In [131]:
# Let's add the __init__()
class A(metaclass=EnforceFieldTypes):

    field_types = {'name':str, 'age':int}

    def __init__(self):
        self.name = "Anil"

    def print(self):
        print("This is a class")

The above doesn't give an error

##### Let's develop the Student class now

In [152]:
class Student(metaclass=EnforceFieldTypes):

    field_types = {
        "name"  : str,
        "age"   : int,
        "gpa"   : float,
        "major" : str
    }

    def __init__(self, name, age, gpa, major):
        print(locals()) # locals() returns a dictionary of the arguemnts that are passed
        

In [154]:
s = Student(name='Ram', age=21, gpa=9, major='AI')

{'self': <__main__.Student object at 0x000001DCB67163F0>, 'name': 'Ram', 'age': 21, 'gpa': 9, 'major': 'AI'}


In [164]:
class Student(metaclass=EnforceFieldTypes):

    field_types = {
        "name"  : str,
        "age"   : int,
        "gpa"   : float,
        "major" : str
    }

    # The question is how do I inject this section from the metaclass
    def __init__(self, name, age, gpa, major):
        for field, expected_type in self.field_types.items():
            value = locals()[field]
            print(value, field, locals()[field])
            if not isinstance(value, expected_type):
                raise TypeError(f"{field} must of type {expected_type}")
            setattr(self, field, value)

    def get_details(self):
        return self.name, self.age, self.gpa, self.major

    def is_honors(self):
        return self.gpa >= 3.5

    def update_gpa(self, new_gpa):
        if not isinstance(new_gpa, float):
            raise TypeError("GPA must be a float")
        elif not (0.0 <= new_gpa <= 4.0):
            raise ValueError("GPA must be between 0.0 and 4.0")
        self.gpa = new_gpa
                            


    

In [166]:
s = Student(name='Ram', age=21, gpa=4.0, major='AI')

Ram name Ram
21 age 21
4.0 gpa 4.0
AI major AI


#### With injection of init

In [172]:
# Meta Class with __init__ injection
class EnforceFieldTypes(type):

    def __new__(cls, name, bases, dct):

        # Extract field_types
        field_types = dct.get('field_types') # dct['field_types']
         
        # Validate the field_types
        if not isinstance(field_types, dict):
            raise TypeError(f"Class '{name}' must define 'field_types' dictionary")

        '''
        # We will also enforce __init__ to be defined in the class
        # Since you're injecting the code, there is not need to validate
        if '__init__' not in dct:
            raise TypeError(f"Class '{name}' must define '__init__' function")
        '''
        # code to inject
        def __init__(self, **kargs):
            for field, expected_type in self.field_types.items():
                if field not in kargs:
                    raise TypeError(f"Missing required field: {field}")
                if not isinstance(kargs[field], expected_type):
                    raise TypeError(f"{field} must of type {expected_type}")
                setattr(self, field, kargs[field])

        # inject the code
        dct['__init__'] = __init__

        return super().__new__(cls, name, bases, dct)

In [174]:
class Student(metaclass=EnforceFieldTypes):

    field_types = {
        "name"  : str,
        "age"   : int,
        "gpa"   : float,
        "major" : str
    }

    def get_details(self):
        return self.name, self.age, self.gpa, self.major

    def is_honors(self):
        return self.gpa >= 3.5

    def update_gpa(self, new_gpa):
        if not isinstance(new_gpa, float):
            raise TypeError("GPA must be a float")
        elif not (0.0 <= new_gpa <= 4.0):
            raise ValueError("GPA must be between 0.0 and 4.0")
        self.gpa = new_gpa
                            

In [178]:
s = Student(name='Ram', age=21, gpa=4.0, major='AI')
s.get_details()

('Ram', 21, 4.0, 'AI')

In [180]:
s = Student(name='Ram', age=2.3, gpa=4.0, major='AI')
s.get_details()

TypeError: age must of type <class 'int'>

In [182]:
try:
    s = Student(name='Ram', age=2.3, gpa=4.0, major='AI')
    s.get_details()
except Exception as e:
    print(e)

age must of type <class 'int'>
